In [7]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [8]:
load_dotenv()  # Carga las variables desde .env

api_key = os.getenv("DEEPSEEK_KEY")

In [9]:
client = OpenAI(
    api_key=os.getenv("DEEPSEEK_KEY"),
    base_url="https://openrouter.ai/api/v1"
)

In [10]:
def query_deepseek(prompt: str, model: str = "deepseek/deepseek-chat-v3-0324:free") -> str:
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role":"user", "content":prompt}],
        temperature=0.6,
        # Muestra la cantidad de output
        max_tokens=4096,
    )
    return resp.choices[0].message.content.strip()

In [12]:
# Carga el notebook activo (puedes ver su nombre en Kaggle)
from langchain_community.document_loaders import NotebookLoader

loader = NotebookLoader("prueba-llm.ipynb", include_outputs=False, remove_newline=True)
docs = loader.load()
print(f"Celdas encontradas: {len(docs)}")

ModuleNotFoundError: No module named 'langchain_community'

Este es el prompt que se debe hacer de manera general

In [ ]:
codigo_original = """
# Carga el notebook activo (puedes ver su nombre en Kaggle)
from langchain_community.document_loaders import NotebookLoader

loader = NotebookLoader("prueba-llm.ipynb", include_outputs=False, remove_newline=True)
docs = loader.load()
print(f"Celdas encontradas: {len(docs)}")
"""

prompt = f"""Reescribe el siguiente código con mejoras de estilo y eficiencia, sin cambiar su funcionalidad:\n{codigo_original},
ademas quiero que todo lo que no este relacionado a codigo lo dejes comentado, incluyendo los cambios realizados"""

respuesta = query_deepseek(prompt)
print(respuesta)


Aquí está el código reescrito con mejoras de estilo y eficiencia, manteniendo la funcionalidad original y comentando los cambios realizados:

```python
# Importación del cargador de notebooks de LangChain
from langchain_community.document_loaders import NotebookLoader

# Se inicializa el cargador con parámetros específicos:
# - Nombre del archivo: "prueba-llm.ipynb"
# - include_outputs=False: no incluye salidas de celdas
# - remove_newline=True: elimina saltos de línea
loader = NotebookLoader(
    file_path="prueba-llm.ipynb",
    include_outputs=False,
    remove_newline=True
)

# Carga los documentos (celdas del notebook) y muestra el conteo
docs = loader.load()
print(f"Celdas encontradas: {len(docs)}")
```

# Mejoras realizadas:
# 1. Formateo más limpio de la inicialización del NotebookLoader
# 2. Parámetro renombrado de "prueba-llm.ipynb" a "file_path" para mayor claridad
# 3. Comentarios más descriptivos y organizados
# 4. Mantenimiento de la funcionalidad original sin cambios
# 5

In [ ]:
from IPython.core.getipython import get_ipython

# Mostrar sugerencia
print("🛠️ Sugerencia del modelo:\n")
print(respuesta)

# Opcional: reemplazar esta celda o copiarla en otra celda
get_ipython().set_next_input(respuesta)

🛠️ Sugerencia del modelo:

Aquí está el código reescrito con mejoras de estilo y eficiencia, manteniendo la funcionalidad original y comentando los cambios realizados:

```python
# Importación del cargador de notebooks de LangChain
from langchain_community.document_loaders import NotebookLoader

# Se inicializa el cargador con parámetros específicos:
# - Nombre del archivo: "prueba-llm.ipynb"
# - include_outputs=False: no incluye salidas de celdas
# - remove_newline=True: elimina saltos de línea
loader = NotebookLoader(
    file_path="prueba-llm.ipynb",
    include_outputs=False,
    remove_newline=True
)

# Carga los documentos (celdas del notebook) y muestra el conteo
docs = loader.load()
print(f"Celdas encontradas: {len(docs)}")
```

# Mejoras realizadas:
# 1. Formateo más limpio de la inicialización del NotebookLoader
# 2. Parámetro renombrado de "prueba-llm.ipynb" a "file_path" para mayor claridad
# 3. Comentarios más descriptivos y organizados
# 4. Mantenimiento de la funcionalid

In [ ]:
def analizar_archivo(filepath: str, pregunta: str):
    with open(filepath, "r", encoding="utf-8") as f:
        contenido = f.read()

    prompt = f"""Tengo este archivo llamado `{filepath}` con el siguiente contenido:

[INICIO DEL ARCHIVO]
{contenido}
[FIN DEL ARCHIVO]

Ahora, por favor responde lo siguiente:
{pregunta}
"""
    respuesta = query_deepseek(prompt)
    print(respuesta)


In [ ]:
analizar_archivo("src/data/data_dictionary.csv", "¿Puedes revisar este archivo y recomendarme su feature engineering?")

Basándome en el diccionario de datos proporcionado, aquí tienes recomendaciones detalladas de feature engineering para optimizar el conjunto de datos:

### 1. **Codificación de Variables Categóricas**
   - **One-Hot Encoding** para variables nominales:
     - `Basic_Demos-Enroll_Season` (4 estaciones)
     - `CGAS-Season`, `Physical-Season`, etc. (patrón repetido de estaciones)
   - **Label Encoding** para variables ordinales con jerarquía:
     - `BIA-BIA_Activity_Level_num` (1=Very Light a 5=Exceptional)
     - `computerinternet_hoursday` (0=Less than 1h/day a 3=More than 3hs/day)
   - **Binary Encoding** para variables binarias:
     - `Basic_Demos-Sex` (ya está codificado como 0/1)
     - Variables de zona de fitness (ej. `FGC-FGC_CU_Zone`: 0/1)

### 2. **Normalización/Estandarización**
   - **StandardScaler** para variables continuas con posibles outliers:
     - `Physical-BMI`, `Physical-Height`, `Physical-Weight`
     - Variables de BIA como `BIA-BIA_Fat`, `BIA-BIA_SMM`
   - **M